<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/Chem_DataExtractor_extracting_a_custom_property.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting a Custom Property

In [2]:
!pip install chemdataextractor

     |████████████████████████████████| 182 kB 4.4 MB/s 
     |████████████████████████████████| 743 kB 44.7 MB/s 
     |████████████████████████████████| 371 kB 41.2 MB/s 
     |████████████████████████████████| 5.6 MB 35.5 MB/s 
     |████████████████████████████████| 3.6 MB 35.6 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=857845 sha256=4765c9a3ed0c6e84f6c9a20908c47b31bb91c4a3969eabb4980136230a27c506
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [4]:
from chemdataextractor import Document
from chemdataextractor.model import Compound
from chemdataextractor.doc import Paragraph, Heading

## read file

In [6]:
from chemdataextractor import Document
f = open('j.jnoncrysol.2017.07.006.xml', 'rb')
doc = Document.from_file(f)

IndexError: ignored

## Example Document

Let's create a simple example document with a single heading followed by a single paragraph:

In [ ]:
d = Document(
    Heading(u'Synthesis of 2,4,6-trinitrotoluene (3a)'),
    Paragraph(u'The procedure was followed to yield a pale yellow solid (b.p. 240 °C)')
)

What does this look like:

In [ ]:
d

<Document: 2 elements>

## Default Parsers

By default, ChemDataExtractor won't extract the boiling point property:

In [ ]:
d.records.serialize()

[{'labels': ['3a'], 'names': ['2,4,6-trinitrotoluene'], 'roles': ['product']}]

## Defining a New Property Model

The first task is to define the schema of a new property, and add it to the `Compound` model:

In [ ]:
from chemdataextractor.model import BaseModel, StringType, ListType, ModelType

class BoilingPoint(BaseModel):
    value = StringType()
    units = StringType()
    
Compound.boiling_points = ListType(ModelType(BoilingPoint))

## Writing a New Parser

Next, define parsing rules that define how to interpret text and convert it into the model:

In [ ]:
import re
from chemdataextractor.parse import R, I, W, Optional, merge

prefix = (R(u'^b\.?p\.?$', re.I) | I(u'boiling') + I(u'point')).hide()
units = (W(u'°') + Optional(R(u'^[CFK]\.?$')))(u'units').add_action(merge)
value = R(u'^\d+(\.\d+)?$')(u'value')
bp = (prefix + value + units)(u'bp')

In [ ]:
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first

class BpParser(BaseParser):
    root = bp

    def interpret(self, result, start, end):
        compound = Compound(
            boiling_points=[
                BoilingPoint(
                    value=first(result.xpath('./value/text()')),
                    units=first(result.xpath('./units/text()'))
                )
            ]
        )
        yield compound


In [ ]:
Paragraph.parsers = [BpParser()]

## Running the New Parser

In [ ]:
d = Document(
    Heading(u'Synthesis of 2,4,6-trinitrotoluene (3a)'),
    Paragraph(u'The procedure was followed to yield a pale yellow solid (b.p. 240 °C)')
)

d.records.serialize()

[{'boiling_points': [{'units': '°C', 'value': '240'}],
  'labels': ['3a'],
  'names': ['2,4,6-trinitrotoluene'],
  'roles': ['product']}]